In [127]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
from numba import njit

In [2]:
import matplotlib
matplotlib.get_backend()

'module://matplotlib_inline.backend_inline'

try with generators https://realpython.com/introduction-to-python-generators/

normalizing only works for linear homog recurrence relation.

get recurrence relation.

numerical integration gets computationally expensive bc a lot of n, alpha, x combinations

so use recurrence. know K0 analytically from integral bc simple to evaluate. 

recurrence relation is linear but not homogeneous. could maybe guess particular solution so that we can use bessel function method.

$\alpha / z$ sets critical value for when to use upwards vs downwards recurrence. set this critical value at 1/2. could reduce this number.

for downward recurrence, use power series to get Kn,max. this is expensive, but it allows us to get many others in the recurrence relation for cheap. cost is upfront. coefficients in power series are small. could use numpy integrator but with this way we can control accuracy by adding more terms. 

there is an analytic solution in terms of a power series. we can use this to check.

can also use scipy integrate to check work.

for homework, calculate n=0 to 10. and for several alpha/z

plot Ks together.

fxn of z alpha n
set epsilon for accuracy. if alpha / z ^ k < epsilon, terminate sum. 
set 1/2 for critical down/up.
if less down
    - taylor expand for KN (large N), then go down. keep going until abs value is small. xn=1 ; for m in large number (x_n / z)/(n+m+1). just a taylor series
    - then can just do downward recursion

if greater up: simple. return Kn

plot for all n from 1 to 10
compare different alpha/z values plotting might be weird bc one might be much larger than others
alpha/z = 0.1 0.2 0.3 0.5



In [149]:
class KnSolver:
    def __init__(self, alpha, z, n_max, epsilon):
        self._alpha = alpha
        self._z = z
        self._epsilon = epsilon
        self._n_max = n_max
        self._critical_ratio = 0.5
        
    def calculate_K_zero(self):
        return (1 / self._alpha) * np.log(1 + self._alpha / self._z)
    
    def _calculate_K_zero_at_z(self, z):
        return (1 / self._alpha) * np.log(1 + self._alpha / z)
    
    
    def upwards_recurrence(self):
        Kn = self.calculate_K_zero()
        
        Kn_history = np.zeros((self._n_max, len(self._z)))
        
        for n in range(self._n_max):
            Kn_history[n, :] = Kn
            Kn = 1/(self._alpha * (n + 1)) - (self._z / self._alpha) * Kn
        
        return Kn_history
    
    
    def solve_Kn_integral(self):
        def integrand(x, z, alpha, n):
            return x**n / (z + alpha * x)
        
        Kn_history = np.zeros((self._n_max, len(self._z)))
    
        for n in range(self._n_max):
            for i, z in enumerate(self._z):    
                Kn_history[n, i] = integrate.quad(integrand, 0, 1, args=(z, self._alpha, n))[0]
        
        return Kn_history
    
    
    def taylor_approximate_Kn(self, n, z):
        
        coefficient = 1
        k = 0
        sum = 0
        max_iter = 100000
        while np.abs(coefficient) > self._epsilon:
            coefficient = (-self._alpha / z)**k 
            sum += coefficient / (n + k + 1)
            k += 1 
            
            if k > max_iter:
                print(f'Warning: exceeding {max_iter} iterations.')
                break
        print(f'Number of iterations: {k}')
        return sum / z
    
    #@staticmethod
    #@njit
    def analytic_solution_Kn(self, n, z):
        
        k = 1
        coefficient = 1
        sum = 0
        max_iter = 100000
        while np.abs(coefficient) > self._epsilon:
            coefficient = (-self._alpha / z)**k 
            sum += coefficient / k
            k += 1
            
            if k > max_iter:
                    print(f'Warning: exceeding {max_iter} iterations.')
                    break
        print(f'Number of iterations: {k}')
        print(sum)
        print(coefficient)

        return (-z/self._alpha)**n * (self._calculate_K_zero_at_z(z) + sum / self._alpha)
            
    def calculate_Kn(self):
        
        # if alpha /z down
        # if alopha / up
        
        raise NotImplementedError
        
    

In [150]:
1e-200

1e-200

In [151]:
solver = KnSolver(0.1, np.array([1, 2, 3, 4]), 10, 1e-2)

In [152]:
solver.taylor_approximate_Kn(2, 4)

Number of iterations: 3


0.08180208333333333

In [153]:
solver.analytic_solution_Kn(2, 4)

Number of iterations: 3
-0.0246875
0.0006250000000000001


0.08180144594263083

In [68]:
Kn_history = solver.upwards_recurrence()

In [69]:
print(Kn_history)

[[0.9531018  0.48790164 0.32789823 0.24692613]
 [0.46898202 0.24196717 0.16305315 0.12295496]
 [0.3101798  0.16065668 0.10840541 0.08180145]
 [0.23153529 0.12019978 0.08117113 0.0612755 ]
 [0.1846471  0.09600442 0.06486623 0.04898017]
 [0.15352901 0.07991151 0.05401319 0.040793  ]
 [0.13137658 0.06843651 0.04627084 0.03494647]
 [0.1148056  0.05984115 0.04044618 0.0307127 ]
 [0.10194398 0.05317703 0.03661465 0.02149212]
 [0.09167135 0.04757043 0.01267156 0.25142629]]


In [70]:
solver.solve_Kn_integral()

array([[0.9531018 , 0.48790164, 0.32789823, 0.24692613],
       [0.46898202, 0.24196717, 0.16305315, 0.12295496],
       [0.3101798 , 0.16065668, 0.10840541, 0.08180145],
       [0.23153529, 0.12019978, 0.08117113, 0.0612755 ],
       [0.1846471 , 0.09600442, 0.06486623, 0.04898018],
       [0.15352901, 0.07991151, 0.05401322, 0.04079292],
       [0.13137658, 0.06843648, 0.04627011, 0.03495001],
       [0.11480561, 0.05984179, 0.04046805, 0.03057083],
       [0.10194391, 0.05316421, 0.0359586 , 0.02716666],
       [0.09167203, 0.04782684, 0.03235318, 0.02444455]])